### Student Information
Name: 徐偉晉

Student ID: 109022127

GitHub ID: Wayne48763

Kaggle name: WayneHsu123

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the DM2023-Lab2-master. You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/t/09b1d0f3f8584d06848252277cb535f2) regarding Emotion Recognition on Twitter by this link https://www.kaggle.com/t/09b1d0f3f8584d06848252277cb535f2. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Dec. 27th 11:59 pm, Wednesday)_. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 31th 11:59 pm, Sunday)__. 

In [33]:
### Begin Assignment Here
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
print(torch.cuda.is_available())

True


# Load Data

In [35]:
# load data from pickle file
df_train = pd.read_pickle('data/train_unbias.pkl')
df_test = pd.read_pickle('data/test.pkl')

In [36]:
df_train.head()

,tweet_id,hashtags,text,identification,emotion,text_clean
0,0x241418,"[facebookdown, instagramisdown, bored]",Facebook's isn't working! Anyone else having t...,train,anger,Facebook's working! Anyone else trouble? #face...
1,0x36ccb2,[],@ASK_des His death really <LH> me off!,train,anger,@ASK_des His death really <LH> off!
2,0x22f974,[shooting],@CNN <LH> Guy's #shooting rampant occurred in ...,train,anger,@CNN <LH> Guy's #shooting rampant occurred Las...
3,0x2edf43,[],It's my pay day and I'm not getting paid <LH>,train,anger,It's pay day I'm getting paid <LH>
4,0x2c3b05,"[PhoenixRally, message, Madness]",#PhoenixRally some dude (#Talkinghead)said the...,train,anger,#PhoenixRally dude (#Talkinghead)said there's ...


In [37]:
# print max length of sentences
print('max length of sentences: ', df_train['text'].str.len().max())

max length of sentences:  158


# Data Preprocessing

In [38]:
from transformers import RobertaTokenizerFast
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [39]:
tokenizer = RobertaTokenizerFast.from_pretrained('cardiffnlp/twitter-roberta-base-2022-154m')

In [40]:
# build label encoder
ohe = OneHotEncoder(sparse_output=True)
ohe.fit(df_train[['emotion']])
print(ohe.categories_)

[array(['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness',
       'surprise', 'trust'], dtype=object)]


In [41]:
df_train['label'] = ohe.transform(df_train[['emotion']]).toarray().tolist()

In [42]:
df_train.head()

,tweet_id,hashtags,text,identification,emotion,text_clean,label
0,0x241418,"[facebookdown, instagramisdown, bored]",Facebook's isn't working! Anyone else having t...,train,anger,Facebook's working! Anyone else trouble? #face...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,0x36ccb2,[],@ASK_des His death really <LH> me off!,train,anger,@ASK_des His death really <LH> off!,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,0x22f974,[shooting],@CNN <LH> Guy's #shooting rampant occurred in ...,train,anger,@CNN <LH> Guy's #shooting rampant occurred Las...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,0x2edf43,[],It's my pay day and I'm not getting paid <LH>,train,anger,It's pay day I'm getting paid <LH>,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,0x2c3b05,"[PhoenixRally, message, Madness]",#PhoenixRally some dude (#Talkinghead)said the...,train,anger,#PhoenixRally dude (#Talkinghead)said there's ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [43]:
# build dataset
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import f1_score
import gc

class TweetDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        return self.df.iloc[index]
    
# train test split
dataset = TweetDataset(df_train, tokenizer, 160)
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
dataset = None
gc.collect()

0

In [44]:
# build dataloader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
print(train_loader)

In [24]:
from transformers import RobertaForSequenceClassification

In [25]:
model = RobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-2022-154m', num_labels=8)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2022-154m and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

In [27]:
EPOCHS = 1

In [26]:
# train model
model.cuda()
model.train()
train_losses = []
val_losses = []
train_accs = []
val_accs = []
train_f1s = []
val_f1s = []

for epoch in range(EPOCHS):
    train_loss = 0
    train_acc = 0
    train_f1 = 0
    val_loss = 0
    val_acc = 0
    val_f1 = 0

    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].cuda().squeeze()
        attention_mask = batch['attention_mask'].cuda()
        labels = batch['labels'].cuda()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += (outputs[1].argmax(1) == labels.argmax(1)).sum().item()
        train_f1 += f1_score(labels.cpu().numpy().argmax(1), outputs[1].argmax(1).cpu().numpy(), average='weighted')
        
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch['input_ids'].cuda().squeeze()
            attention_mask = batch['attention_mask'].cuda()
            labels = batch['labels'].cuda()

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]

            val_loss += loss.item()
            val_acc += (outputs[1].argmax(1) == labels.argmax(1)).sum().item()
            val_f1 += f1_score(labels.cpu().numpy().argmax(1), outputs[1].argmax(1).cpu().numpy(), average='weighted')
    
    train_loss /= len(train_loader)
    train_acc /= len(train_dataset)
    train_f1 /= len(train_loader)
    val_loss /= len(val_loader)
    val_acc /= len(val_dataset)
    val_f1 /= len(val_loader)

    train_losses.append(train_loss)
    train_accs.append(train_acc)
    train_f1s.append(train_f1)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    val_f1s.append(val_f1)

    print('Epoch: ', epoch+1)
    print('Train loss: ', train_loss)
    print('Train acc: ', train_acc)
    print('Train f1: ', train_f1)
    print('Val loss: ', val_loss)
    print('Val acc: ', val_acc)
    print('Val f1: ', val_f1)

  0%|          | 0/1994 [00:00<?, ?it/s]

Epoch:  1
Train loss:  4.720054744388455e-05
Train acc:  1.1757881700032921e-05
Train f1:  5.2596926854382e-06
Val loss:  0.00018601621703852862
Val acc:  0.00010973850881043457
Val f1:  6.26470916016022e-05


In [ ]:
torch.save(model, '/model/roberta_base_2022_154m.pt')

In [ ]:
model = torch.load('/model/roberta_base_2022_154m.pt')
model.cuda()

In [32]:
preds = []

with torch.no_grad():
    for row in tqdm(df_test.iterrows(), total=len(df_test)):
        id = row[1]['identification']
        text = row[1]['text']
        inputs = tokenizer(text, padding='max_length', truncation=True, max_length=160, return_tensors='pt')
        inputs = {k: v.cuda() for k, v in inputs.items()}
        outputs = model(**inputs)
        label = ohe.inverse_transform(outputs[0].cpu().numpy())
        preds.append([id, label])

  1%|          | 3407/411972 [00:35<1:10:28, 96.62it/s] 


KeyboardInterrupt: 